In [1]:
import gradio as gr
import language_tool_python
import docx
import PyPDF2
from docx import Document

# Initialize the grammar tool
tool = language_tool_python.LanguageTool('en-US')


# Function to extract text from uploaded files
def extract_text_from_file(file):
    text = ""
    if file.name.endswith(".docx"):
        doc = docx.Document(file)
        for para in doc.paragraphs:
            text += para.text + "\n"
    elif file.name.endswith(".pdf"):
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    else:
        text = file.read().decode("utf-8")
    return text


# Function for correcting direct text input
def correct_text(text):
    matches = tool.check(text)
    corrected = language_tool_python.utils.correct(text, matches)
    summary = f"🔍 Grammar issues found: {len(matches)}"
    return corrected, summary


# Function for file correction + summary + downloadable output
def correct_file(file):
    raw_text = extract_text_from_file(file)
    matches = tool.check(raw_text)
    corrected = language_tool_python.utils.correct(raw_text, matches)

    # Save corrected text to Word file
    doc = Document()
    doc.add_paragraph(corrected)
    doc_path = "Corrected_Output.docx"
    doc.save(doc_path)

    summary = f"📄 File processed.\n🔍 Grammar issues found: {len(matches)}"

    return corrected, summary, doc_path


# Gradio Blocks UI
with gr.Blocks(title="Grammar Error Detection and Correction") as app:
    gr.Markdown("## 📝 Grammar Error Detection & Correction\nThis app supports live typing and file upload correction.")

    with gr.Tab("1️⃣ Direct Text Input"):
        input_text = gr.Textbox(label="Enter text", lines=5, placeholder="Type your sentence here...")
        corrected_output = gr.Textbox(label="Corrected Text")
        summary_output = gr.Textbox(label="Grammar Summary")
        btn1 = gr.Button("Correct")
        btn1.click(fn=correct_text, inputs=input_text, outputs=[corrected_output, summary_output])

    with gr.Tab("2️⃣ File Upload"):
        file_input = gr.File(label="Upload PDF / DOCX / TXT")
        corrected_file_output = gr.Textbox(label="Corrected Text")
        file_summary_output = gr.Textbox(label="Grammar Summary")
        download_btn = gr.File(label="Download Corrected DOCX")
        btn2 = gr.Button("Process File")
        btn2.click(fn=correct_file, inputs=file_input, outputs=[corrected_file_output, file_summary_output, download_btn])

app.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
